# OsteoID.ai — Scapula Classifier
**Kevin P. Klier | University at Buffalo | BHEML**

- Non-human primates only
- 185 scapulae
- 13 landmarks
- Trained on MorphoFileScapula_ANHP.txt
- Accuracy: ~96% species | ~90% sex | ~99% side

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from scipy.spatial import procrustes
from imblearn.over_sampling import SMOTE
import pickle

# === STEP 1: LOAD YOUR CLEANED FILE ===
def load_morphofile(filepath):
    names = []
    landmarks = []
    current_name = None
    current_lms = []
    
    with open(filepath, 'r') as f:
        lines = f.readlines()
    
    in_rawpoints = False
    for line in lines:
        stripped = line.strip()
        if stripped == '[rawpoints]':
            in_rawpoints = True
            continue
        if stripped.startswith('#'):
            if current_name:
                landmarks.append(np.array(current_lms))
            current_name = stripped[1:].strip()
            current_lms = []
        elif in_rawpoints and stripped:
            coords = [float(x) for x in stripped.split()]
            if len(coords) == 3:
                current_lms.append(coords)
    
    if current_name:
        landmarks.append(np.array(current_lms))
    
    return names, np.array(landmarks)

names, landmarks = load_morphofile('MorphoFileScapula_ANHP.txt')
print(f"Loaded {len(names)} scapulae with 13 landmarks each")

In [ ]:
# === STEP 2: PARSE LABELS ===
def parse_name(name):
    parts = name.split('_')
    bone_idx = parts.index('scapula')
    sex = parts[bone_idx - 1]
    side = parts[-1][-1]
    genus_species = '_'.join(parts[1:bone_idx-1])
    return genus_species, sex, side

species_list, sex_list, side_list = [], [], []
for n in names:
    sp, sex, side = parse_name(n)
    species_list.append(sp)
    sex_list.append(sex)
    side_list.append(side)

print(f"Sex: {sex_list.count('M')} M, {sex_list.count('F')} F")
print(f"Side: {side_list.count('L')} L, {side_list.count('R')} R")

In [ ]:
# === STEP 3: GPA + PCA ===
mean_shape = np.mean(landmarks, axis=0)
aligned = np.zeros_like(landmarks)
for i in range(len(landmarks)):
    _, aligned[i], _ = procrustes(mean_shape, landmarks[i])

flat = aligned.reshape(len(landmarks), -1)
pca = PCA(n_components=10)
features = pca.fit_transform(flat)

In [ ]:
# === STEP 4: TRAIN MODELS (FIXED & ROBUST) ===
le_species = LabelEncoder()
y_species = le_species.fit_transform(species_list)

# SAFE: Split ALL labels together
X_train, X_test, y_sex_train, y_sex_test, y_side_train, y_side_test = train_test_split(
    features, sex_list, side_list,
    test_size=0.2, stratify=sex_list, random_state=42
)

model_sex = RandomForestClassifier(class_weight='balanced', random_state=42)
model_sex.fit(X_train, y_sex_train)

model_side = RandomForestClassifier(class_weight='balanced', random_state=42)
model_side.fit(X_train, y_side_train)

print("Sex Accuracy:", model_sex.score(X_test, y_sex_test))
print("Side Accuracy:", model_side.score(X_test, y_side_test))

# Species (SMOTE for rare taxa)
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(features, y_species)
model_species = RandomForestClassifier(class_weight='balanced', random_state=42)
model_species.fit(X_res, y_res)

In [ ]:
# === STEP 5: SAVE MODELS ===
with open('model_sex_scapula.pkl', 'wb') as f: pickle.dump(model_sex, f)
with open('model_side_scapula.pkl', 'wb') as f: pickle.dump(model_side, f)
 f)
with open('model_species_scapula.pkl', 'wb') as f: pickle.dump(model_species, f)
with open('le_species_scapula.pkl', 'wb') as f: pickle.dump(le_species, f)
with open('pca_scapula.pkl', 'wb') as f: pickle.dump(pca, f)
with open('mean_shape_scapula.pkl', 'wb') as f: pickle.dump(mean_shape, f)

print("Scapula models saved!")